demo对应的材料
+ https://mlc.ai/zh/chapter_tensor_program/case_study.html

In [1]:
import numpy as np
import tvm
from tvm.ir.module import IRModule
from tvm.script import tir as T

来看一个例子，对于两个大小为128x128的矩阵A和B，进行如下两步张量运算
$$
Y_{i, j} = \sum_k A_{i, k} \times B_{k, j} \\
C_{i, j} = \mathbb{relu}(Y_{i, j}) = \mathbb{max}(Y_{i, j}, 0)
$$
使用numpy数组中的计算实现这两个操作

In [3]:
dtype = "float32"
a_np = np.random.rand(128, 128).astype(dtype)
b_np = np.random.rand(128, 128).astype(dtype)
# a @ b is equivalent to np.matmul(a, b)
c_mm_relu = np.maximum(a_np @ b_np, 0)

下面是实现mm_relu操作的一种方式

In [4]:
def lnumpy_mm_relu(A: np.ndarray, B: np.ndarray, C: np.ndarray):
    # 使用的缓冲区
    Y = np.empty((128, 128), dtype="float32")    
    for i in range(128):
        for j in range(128):
            for k in range(128):
                if k == 0:
                    Y[i, j] = 0
                Y[i, j] = Y[i, j] + A[i, k] * B[k, j]
    
    for i in range(128):
        for j in range(128):
            C[i, j] = max(Y[i, j], 0)

In [5]:
c_np = np.empty((128, 128), dtype=dtype)
lnumpy_mm_relu(a_np, b_np, c_np)
np.testing.assert_allclose(c_mm_relu, c_np, rtol=1e-5)

上述函数展示了这些计算的实现中使用的所有可能的元素
+ 多维缓冲区
+ 在数组维度上的循环
+ 在循环下执行的计算语句
下面是一种TVMScript的实现，这是一种嵌入在Python AST中的特定领域的方言

In [10]:
@tvm.script.ir_module
class MyMudule:
    @T.prim_func
    def mm_relu(A: T.Buffer[(128, 128), "float32"],
                B: T.Buffer[(128, 128), "float32"],
                C: T.Buffer[(128, 128), "float32"],):
        T.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})
        # 使用的缓冲区
        Y = T.alloc_buffer((128, 128), dtype="float32")
        for i, j, k in T.grid(128, 128, 128):
            with T.block("Y"):
                # 128指明外面的循环迭代器需要128
                vi = T.axis.spatial(128, i)
                vj = T.axis.spatial(128, j)
                vk = T.axis.reduce(128, k)
                # 在初始化阶段，将Y置为0
                with T.init():
                    Y[vi, vj] = T.float32(0)
                Y[vi, vj] = Y[vi, vj] + A[vi, vk] * B[vk, vj]
        for i, j in T.grid(128, 128):
            with T.block("C"):
                vi = T.axis.spatial(128, i)
                vj = T.axis.spatial(128, j)
                C[vi, vj] = T.max(Y[vi, vj], T.float32(0))

基本思想，使用`block`表示出可能的计算单元
+ `spatial axis`称为**空间轴**，会直接出现在计算结果的某一个维度上
+ `reduce axis`称为**规约轴**，不会直接出现在结果的某个维度上，但是为了得到结果需要遍历其
为什么要声明`spatial axis`和`reduce axis`，为了**给之后的分析和优化提供更多的信息**，`block`是一个自洽的计算，表示需要做很么样的计算，以及循环、外部的数据以来等信息，为后续做优化做准备
+ 这些附加信息也有助于我们进行机器学习编译分析，大部分时候在空间轴上做并行化，在规约轴上进行并行化将需要特定的策略

一些语法糖，可以直接将其与外部的迭代器绑定

In [11]:
@tvm.script.ir_module
class MyModuleWithAxisRemapSugar:
    @T.prim_func
    def mm_relu(A: T.Buffer[(128, 128), "float32"],
                B: T.Buffer[(128, 128), "float32"],
                C: T.Buffer[(128, 128), "float32"]):
        T.func_attr({"global_symbol": "mm_relu", "tir.noalias": True})
        Y = T.alloc_buffer((128, 128), dtype="float32")
        for i, j, k in T.grid(128, 128, 128):
            with T.block("Y"):
                vi, vj, vk = T.axis.remap("SSR", [i, j, k])
                with T.init():
                    Y[vi, vj] = T.float32(0)
                Y[vi, vj] = Y[vi, vj] + A[vi, vk] * B[vk, vj]
        for i, j in T.grid(128, 128):
            with T.block("C"):
                vi, vj = T.axis.remap("SS", [i, j])
                C[vi, vj] = T.max(Y[vi, vj], T.float32(0))